In [1]:
!pip install datasets transformers peft

In [2]:
from datasets import load_dataset

dataset = load_dataset("Msobhi/virgool_62k")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [3]:
dataset["train"][10]

{'title': 'دزدی که پاسبان شد',
 'text': 'دزدی که پاسبان شد\nسلام بانو\nسلام به ابرهای پربار و بارانزای پیشانی بلندتان.\nپنجره\u200cها همه خیسند اما از صدای نگاه و عطر "آهو فراموشکار کن " نفسهایتان خبری نیست.\nپنجره\u200cها را بسته نگاه ندارید، این هوای چله اول زمستان آنقدرها هم که پیشینیان گفته\u200cاند دزد نیست.\nشاید هم پیشتر دزد بوده و حالا توبه کرده باشد.\nدزد هم بود دزدهای قدیم، حول و حوش اورسی\u200cهای بزرگ و باز هشتی اندرونی حیاطهای بیدمجنون دار خانه\u200cهای دختر دم\u200cبخت دار می\u200cگشت و عطر تن و گیسوی آب و شانه خورده را مثل نان خشک می\u200cزد در آب حوض شیشه\u200cهای رنگی رنگی پنجدری\u200cها و می\u200cبردشان برای دلباختگان نشسته پای دیوار کاهگلی کوچه باغ منتهی به بازارچه؛ از دارا می\u200cزد و به ندار می\u200cداد.\nنداری هم اگر دستش بگویی نگویی به دهانش می\u200cرسید و چرب بود، سبیل دزد را چرب کرده و دمش را می\u200cدید و عطر گیسوی دست نخورده و پیراهن دکمه نبسته گیرش می\u200cآمد.\nنوبت به ما که می\u200cرسد آسمان که می\u200cتپد هیچ، دزدها هم تائب می\u200cشوند، پاسبانی پیشه می\

In [4]:
from datasets import load_dataset, ClassLabel, Dataset, DatasetDict
import random
import pandas as pd
from IPython.display import display, HTML
from sklearn.model_selection import train_test_split

In [5]:
# Filter out records with 'خودشناسی' tag
def filter_by_tag(example):
    return 'خودشناسی' in example['tags']

filtered_dataset = dataset['train'].filter(filter_by_tag)

# Remove specified columns
columns_to_remove = ['title', 'tags', 'likes', 'replies', 'reading_time', 'user_id', 'url']
filtered_dataset = filtered_dataset.remove_columns(columns_to_remove)

# Convert dataset to pandas DataFrame
df = pd.DataFrame(filtered_dataset)

# Split the DataFrame into train and test sets
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

# Convert DataFrames back to Hugging Face Datasets
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

# Create a DatasetDict
split_dataset = DatasetDict({
    'train': train_dataset,
    'test': test_dataset
})


In [6]:
def show_filtered_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [7]:
# Show examples from train and test sets
print("Train set examples:")
show_filtered_elements(split_dataset['train'], num_examples=10)

print("Test set examples:")
show_filtered_elements(split_dataset['test'], num_examples=10)

Train set examples:


Test set examples:


In [8]:
print(split_dataset)

DatasetDict({
    train: Dataset({
        features: ['text', '__index_level_0__'],
        num_rows: 400
    })
    test: Dataset({
        features: ['text', '__index_level_0__'],
        num_rows: 101
    })
})


In [9]:
# Assuming 'split_dataset' is a DatasetDict object
for split in split_dataset:
    if '__index_level_0__' in split_dataset[split].column_names:
        split_dataset[split] = split_dataset[split].remove_columns('__index_level_0__')

In [10]:
print(split_dataset)

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 400
    })
    test: Dataset({
        features: ['text'],
        num_rows: 101
    })
})


In [11]:
from transformers import AutoTokenizer

model_checkpoint = "HooshvareLab/bert-fa-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

max_length = 128
# Function to tokenize and truncate the text
def tokenize_and_truncate(examples):
    return tokenizer(examples["text"], truncation=True, max_length=max_length)


tokenized_datasets = split_dataset.map(tokenize_and_truncate, batched=True, num_proc=4, remove_columns=["text"])

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Map (num_proc=4):   0%|          | 0/400 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/101 [00:00<?, ? examples/s]

In [12]:
# Verify the lengths of the sequences
def check_sequence_lengths(dataset, max_length):
    lengths = [len(input_ids) for input_ids in dataset['input_ids']]
    return all(length <= max_length for length in lengths)

# Check the lengths of the sequences in train and test sets
train_lengths_ok = check_sequence_lengths(tokenized_datasets['train'], max_length)
test_lengths_ok = check_sequence_lengths(tokenized_datasets['test'], max_length)

if train_lengths_ok and test_lengths_ok:
    print("All sequences are within the specified maximum length.")
else:
    print("Some sequences exceed the specified maximum length.")


All sequences are within the specified maximum length.


In [13]:
from transformers import AutoModelForMaskedLM
model = AutoModelForMaskedLM.from_pretrained(model_checkpoint)

Some weights of the model checkpoint at HooshvareLab/bert-fa-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [14]:
from peft import LoraConfig, get_peft_model

lora_config = LoraConfig(
    r=8,  # Rank of the low-rank approximation
    lora_alpha=32,  # Scaling factor
    lora_dropout=0.1,  # Dropout rate
    target_modules=["query", "key"],  # Modules to apply LoRA
)

model = get_peft_model(model, lora_config)

No CUDA runtime is found, using CUDA_HOME='/usr/local/cuda'


In [15]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    # per_device_train_batch_size=8,
    learning_rate=2e-5,
    logging_dir='./logs',
    eval_strategy = "epoch",
    weight_decay=0.01,
)

In [16]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [17]:
from transformers import DataCollatorForLanguageModeling
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15)

In [18]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['test'],
    data_collator=data_collator,
)


In [19]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,No log,No log
2,No log,No log
3,No log,No log


TrainOutput(global_step=150, training_loss=2.706806437174479, metrics={'train_runtime': 2558.5782, 'train_samples_per_second': 0.469, 'train_steps_per_second': 0.059, 'total_flos': 79297265664000.0, 'train_loss': 2.706806437174479, 'epoch': 3.0})

In [21]:
# Save the fine-tuned model
model.save_pretrained("/content/drive/MyDrive/fine-tuning-project/task4")
tokenizer.save_pretrained("/content/drive/MyDrive/fine-tuning-project/task4")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


('/content/drive/MyDrive/fine-tuning-project/task4/tokenizer_config.json',
 '/content/drive/MyDrive/fine-tuning-project/task4/special_tokens_map.json',
 '/content/drive/MyDrive/fine-tuning-project/task4/vocab.txt',
 '/content/drive/MyDrive/fine-tuning-project/task4/added_tokens.json',
 '/content/drive/MyDrive/fine-tuning-project/task4/tokenizer.json')